In [ ]:
using Logging
logger = ConsoleLogger(stdout)
# debuglogger = ConsoleLogger(stderr, Logging.Debug)
global_logger(logger)
using JLD

using Random
rng = MersenneTwister(1234)
import Dates

In [ ]:
using Plots
pyplot()

In [ ]:
include("utils/misc.jl")
using .MiscUtils
include("particle_1D/plot_game_1D.jl")
using .PlotGame1DUtils
include("particle_1D/plot_results_1D.jl")
using .PlotRes1D
include("particle_1D/CPMP_bilin_game_1D.jl")
using .CPMPBilinGame1D
include("particle_1D/game_theory_contdom_1D.jl")
using .GameTheoryContdom1DUtils
include("particle_1D/lyapunov_1D.jl")
using .Lyapunov1DUtils

In [ ]:
ts = Dates.now()

alpha = 1
scaling = 1.7 # this is about right for alpha between 0.2 and 4, but there doesnt seem one value for `scaling` that works all the time...

## algo parameters
T = Int(floor(1500 / (alpha^scaling)))
mx = 15 # nb particles
my = 16
eta0_wx = 1e-1 * alpha # initial stepsize
eta0_wy = 1e-1 * alpha
eta0_x  = 1e-2 * alpha
eta0_y  = 1e-2 * alpha
init_pos = "grid_unif" # "iid_unif" or "grid_unif"
extrasteps = 2 # extrasteps=1: CP-MDA, extrasteps=2: CP-MP

## random payoff matrix parameters
YPhsieh_epsilon = 1.

## plotting parameters
deltax = 1e-4 # discretization step when computing (approximation of) global NI error
deltay = 1e-4
evalNIevery    = Int(floor(T/50))
evallocNIevery = Int(floor(T/50))
evalVevery     = Int(floor(T/50))
plotiterevery  = Int(floor(T/50))
xmin, xmax = -2., 2.
ymin, ymax = -2., 2.

## logging
ts_fsfriendly = Dates.format(ts, "yyyy-mm-ddTHHMMSS") # filesystem-friendly string for ts
resultdir = mkpath("results/particle__YPhsieh$(YPhsieh_epsilon)__mx$(mx)y$(my)__extrasteps$(extrasteps)__$(ts_fsfriendly)")

logfile = "$resultdir/log.txt"
touch(logfile)
open(logfile, "a") do f
    write(f, "T=$T\n")
    write(f, "mx=$mx\n")
    write(f, "my=$my\n")
    write(f, "eta0_wx=$(eta0_wx)\n")
    write(f, "eta0_wy=$(eta0_wy)\n")
    write(f, "eta0_x=$(eta0_x)\n")
    write(f, "eta0_y=$(eta0_y)\n")
    write(f, "init_pos=$(init_pos)\n")
    write(f, "extrasteps=$(extrasteps)\n")
    write(f, "YPhsieh_epsilon=$(YPhsieh_epsilon)\n")
end

In [ ]:
## instantiate game
phi(x) = x^4/4 - x^2/2
function gfun_centered(x, y)
    return x*y + YPhsieh_epsilon*( phi(x) - phi(y) )
end
# gfun(x, y) = gfun_centered(x-0.5, y-0.5)
gfun = gfun_centered
heatmap_gfun_1D(gfun; xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)

In [ ]:
## solve game
#==============#
copies_wx, copies_x, copies_wy, copies_y = run_CPMP(gfun, T, mx, my, eta0_wx, eta0_x, eta0_wy, eta0_y, init_pos; 
    extrasteps=extrasteps, 
    true_prox=true,
    rng=rng,
    floorto01=false,
    xinitmin=xmin, xinitmax=xmax, yinitmin=ymin, yinitmax=ymax)
avg_wx, avg_x, avg_wy, avg_y = avg_iter(copies_wx, copies_x, copies_wy, copies_y)
save("$resultdir/iterates.jld", "copies_wx", copies_wx, "copies_x", copies_x, "copies_wy", copies_wy, "copies_y", copies_y)

In [ ]:
# Use the last iterate as reference point (estimate of the true MNE)
wx0, x0, wy0, y0 = copies_wx[:,T+1], copies_x[:,T+1], copies_wy[:,T+1], copies_y[:,T+1]
nierr0 = glob_NI_err(gfun, wx0, x0, wy0, y0; deltax=deltax, deltay=deltay, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)
gval0 = payoff(gfun, wx0, x0, wy0, y0)
open(logfile, "a") do f
    write(f, "Using last iterate as reference point (wx0, x0, wy0, y0) with NI error $(nierr0)\n")
    write(f, "Payoff at reference point: gval0=$(gval0)\n")
end
nierr0, gval0

In [ ]:
## compute and plot optimality metrics
#==============#
### compute and plot (global) NI error of iterates
plt_NI, plt_NI_log, nierrs = plot_NI_err(gfun, T, copies_wx, copies_x, copies_wy, copies_y, avg_wx, avg_x, avg_wy, avg_y,
    resultdir, logfile, evalNIevery;
    deltax=deltax, deltay=deltay, skip_avg=true,
    xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)
save("$resultdir/nierrs__every$(evalNIevery).jld", "nierrs", nierrs)
plt_NI_log

In [ ]:
### compute and plot local NI error of iterates
plt_locNI, plt_locNI_log, locnierrs = plot_locNI_err(gfun, T, copies_wx, copies_x, copies_wy, copies_y, avg_wx, avg_x, avg_wy, avg_y,
    resultdir, logfile, evallocNIevery)
plt_locNI_log

In [ ]:
## plot first variations at reference point
#==============#
plt_firstvar_x, plt_firstvar_y = plot_firstvars(gfun, wx0, x0, wy0, y0, gval0,
    resultdir;
    xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)
plt_firstvar_x

In [ ]:
## plot iterations
#==============#
pltt, _, _ = plot_iter_1D(gfun, wx0, x0, wy0, y0; xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)
fn = "$resultdir/last_iter.png"
savefig(pltt, fn)
pltt

In [ ]:
plot_iters(gfun, T, copies_wx, copies_x, copies_wy, copies_y, avg_wx, avg_x, avg_wy, avg_y,
    resultdir,
    plotiterevery;
    xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)

In [ ]:
## plot lyapunov function
#==============#
### Get aggregated representation of the reference point (ideally, should check manually!)
wxstar, xstar, mxstar = aggregate_particles_1D(wx0, x0; thresh_pos=1e-5, torus=false)
wystar, ystar, mystar = aggregate_particles_1D(wy0, y0; thresh_pos=1e-5, torus=false)
dxstar_min = minimum([abs(xstar[I]-xstar[J]) for I=1:mxstar for J=1:I-1])
dystar_min = minimum([abs(ystar[I]-ystar[J]) for I=1:mystar for J=1:I-1])
open(logfile, "a") do f
    write(f, "mxstar, mystar, dxstar_min, dystar_min: $mxstar, $mystar, $dxstar_min, $dystar_min\n")
end
mxstar, mystar, dxstar_min, dystar_min

In [ ]:
alpha_x = 3
alpha_y = 3
lambda_x = 3 # (must tweak manually! paper uses lambda_x = eta_x^{-1/6} for proofs)
lambda_y = 3
tau_x = 1e-2
tau_y = 1e-2

In [ ]:
# ### compute and plot weight and position and total lyapunov function of iterates
# plt_V_x, plt_V_x_log, Vweis_x, Vposs_x, Vtots_x = plot_V(gfun, T, copies_wx, copies_x, avg_wx, avg_x,
#     wxstar, xstar,
#     eta0_wx, eta0_x,
#     resultdir, logfile, "x",
#     evalVevery;
#     alpha=alpha_x, lambda=lambda_x, tau=tau_x)

In [ ]:
# plt_V_y, plt_V_y_log, Vweis_y, Vposs_y, Vtots_y = plot_V(gfun, T, copies_wy, copies_y, avg_wy, avg_y,
#     wystar, ystar,
#     eta0_wy, eta0_y,
#     resultdir, logfile, "y",
#     evalVevery;
#     alpha=alpha_y, lambda=lambda_y, tau=tau_y)

In [ ]:
plt_Vxy, plt_Vxy_log, Vweis_xy, Vposs_xy, Vtots_xy = plot_V_xy(gfun, T, copies_wx, copies_x, copies_wy, copies_y, avg_wx, avg_x, avg_wy, avg_y,
    wxstar, xstar, wystar, ystar, 
    eta0_wx, eta0_x, eta0_wy, eta0_y,
    resultdir, logfile,
    evalVevery;
    alpha_x=alpha_x, lambda_x=lambda_x, tau_x=tau_x,
    alpha_y=alpha_y, lambda_y=lambda_y, tau_y=tau_y)
plt_Vxy_log